# < 알파벳을 기준으로 그룹별로 정렬하기 >
    * 전달받은 데이터는 외국인 셀럽들의 데이터이다. 회사에서 요구한 것은 각 셀럽들의 성이 아닌 이름의 맨 앞 알파벳을 기준으로 그룹을 지어달라는 것!
예 : Amanda Seyfried 는 Actor 그룹의  'S'그룹

1) 성이 아닌 이름을 기준으로 할 것
    - 예외 케이스 고려하기 ( 미들네임 or '.'이 이름에 있는 경우 등)

2) 정렬 기준은 그룹 -> 이름의 알파벳. 각각 인물들의 직업이 우선순위로 그룹화 되어있어야 함



In [1]:
import pandas as pd
import numpy as np
import re

# 전달받은 data set
data = pd.read_excel("/Users/myhellebs/Downloads/키워드 카테고리 구분 작업.xlsx") 
data

,ks_pk,keyword_name,kl_name_en,type
0,817,Mark Hamill,Mark Hamill,Actor
1,818,Harrison Ford,Harrison Ford,Actor
2,819,Peter Cushing,Peter Cushing,Actor
3,820,Anthony Daniels,Anthony Daniels,Actor
4,821,Bob Peterson,Bob Peterson,Actor
...,...,...,...,...
6845,7690,Susanna Fogel,Susanna Fogel,Director : Female
6846,7691,Julia Ducournau,Julia Ducournau,Director : Female
6847,7692,Lucia Aniello,Lucia Aniello,Director : Female
6848,7693,Stella Meghie,Stella Meghie,Director : Female


## < 기준 및 예외 케이스 정하기 >
    - 성 + 이름(두개의 단어)으로 되어 있는 사람은 이름의 맨 앞글자 대문자만 추출
    - 미들네임이 있는 경우에는 맨 뒤에 있는 이름으로 선택. 예)Jennifer Yuh Nelson = N그룹
    - 이름에 Junior or Jr가 있는 경우에는, jr바로 이전 단어를 이름으로 선택. 예)John Gallagher Jr. = G그룹
    - 이름에 특수문자('.','-' etc)가 있는건 특이 케이스 이므로 주의해서 따로 확인할 것

In [2]:
global check_list
check_list = []

def Upper_Group(x):
    
    unit = x.split(" ")
    Alphabet = re.compile('[A-Z]') # 대문자만 추출  
    sc = re.compile("[^a-zA-Z\s]") # 공백을 제외한 특수문자가 있는 이름들의 확인을 위한 정규식
    
    upper = Alphabet.findall(x)
    
    try :  
        if ('Jr.' in unit) or ('Junior' in unit):
            return upper[-2]
        
        else : # stereo type의 모든 이름(성 + 이름)
            return upper[-1] 
        
    finally :     
        if sc.findall(x): # 특수문자가 있으면
            check_list.append(x)    


In [3]:
# 이름의 대문자를 추출한 새로운 칼럼 생성
data['upper'] = data['keyword_name'].apply(Upper_Group)
data

,ks_pk,keyword_name,kl_name_en,type,upper
0,817,Mark Hamill,Mark Hamill,Actor,H
1,818,Harrison Ford,Harrison Ford,Actor,F
2,819,Peter Cushing,Peter Cushing,Actor,C
3,820,Anthony Daniels,Anthony Daniels,Actor,D
4,821,Bob Peterson,Bob Peterson,Actor,P
...,...,...,...,...,...
6845,7690,Susanna Fogel,Susanna Fogel,Director : Female,F
6846,7691,Julia Ducournau,Julia Ducournau,Director : Female,D
6847,7692,Lucia Aniello,Lucia Aniello,Director : Female,A
6848,7693,Stella Meghie,Stella Meghie,Director : Female,M


In [4]:
# 특수문자가 있던 이름들을 조회
data[data['keyword_name'].isin(check_list)].iloc[600:-1] 
# ks_pk = 1012 , 1850 ,1988, 3075

,ks_pk,keyword_name,kl_name_en,type,upper
6495,7340,Mark L. Lester,Mark L. Lester,Director : Male,L
6497,7342,Baltasar Kormákur,Baltasar Kormákur,Director : Male,K
6505,7350,Jean-Marc Vallée,Jean-Marc Vallée,Director : Male,V
6507,7352,David M. Evans,David M. Evans,Director : Male,E
6521,7366,Malcolm D. Lee,Malcolm D. Lee,Director : Male,L
6534,7379,Charles Herman-Wurmfeld,Charles Herman-Wurmfeld,Director : Male,W
6537,7382,Jon S. Baird,Jon S. Baird,Director : Male,B
6539,7384,Gavin O'Connor,Gavin O'Connor,Director : Male,C
6541,7386,Rodrigo García,Rodrigo García,Director : Male,G
6560,7405,Paolo Virzì,Paolo Virzì,Director : Male,V


In [5]:
# ks_pk = 1012 , 1850 ,1988, 3075 - 특이 데이터 확인 후 수정하기
check = [1012 , 1850 ,1988, 3075]
data[data['ks_pk'].isin(check)]

,ks_pk,keyword_name,kl_name_en,type,upper
190,1012,Rade Šerbedžija,Rade Šerbedžija,Actor,R
1020,1850,Soon-Tek Oh,Soon-Tek Oh,Actor,O
1156,1988,John C. McGinley,John C. McGinley,Actor,G
2240,3075,Louis C.K.,Louis C.K.,Actor,K


In [6]:
# 특이 데이터의 알파벳 그룹이 수정됐음을 확인
data.loc[data.ks_pk == 1012, 'upper'] = "S"
data.loc[data.ks_pk == 1850, 'upper'] = "S"
data.loc[data.ks_pk == 1988, 'upper'] = "M"
data.loc[data.ks_pk == 3075, 'upper'] = "L"

data[data['ks_pk'].isin(check)]

,ks_pk,keyword_name,kl_name_en,type,upper
190,1012,Rade Šerbedžija,Rade Šerbedžija,Actor,S
1020,1850,Soon-Tek Oh,Soon-Tek Oh,Actor,S
1156,1988,John C. McGinley,John C. McGinley,Actor,M
2240,3075,Louis C.K.,Louis C.K.,Actor,L


In [7]:
alphabet_group = list(data['upper'].unique())
alphabet_group = sorted(alphabet_group)
type_group = list(data['type'].unique())

In [9]:
final = []

for job in type_group :
    
    for alpha in alphabet_group :
        
        pk_list = list(data[(data['type'] == job) & (data['upper'] == alpha)]['ks_pk'])
        pk_list = str(pk_list).replace("[","").replace("]","")
        
        names = list(data[(data['type'] == job) & (data['upper'] == alpha)]['keyword_name'])
        names = str(names).replace("[","").replace("]","").replace("\'","")
        
        result = {'type':job ,'upper':alpha, 'pk_num':pk_list, 'celeb':names}
        
        final.append(result)
        

In [12]:
# 최종 결과
pd.DataFrame(final)

,type,upper,pk_num,celeb
0,Actor,A,"836, 943, 955, 971, 984, 991, 992, 1019, 1023,...","Alfred Abel, Dan Aykroyd, John Ashton, Ben Aff..."
1,Actor,B,"823, 829, 847, 849, 857, 878, 897, 903, 906, 9...","Albert Brooks, Sean Bean, Orlando Bloom, Kenny..."
2,Actor,C,"819, 854, 863, 864, 866, 904, 907, 908, 947, 9...","Peter Cushing, David Carradine, Dan Castellane..."
3,Actor,D,"820, 841, 873, 934, 1005, 1054, 1060, 1067, 10...","Anthony Daniels, Johnny Depp, Keir Dullea, Joh..."
4,Actor,E,"859, 879, 884, 935, 1025, 1065, 1114, 1127, 11...","Clint Eastwood, Emilio Echevarría, Eminem, Den..."
...,...,...,...,...
99,Director : Female,V,"7668, 7676, 7679","Agnieszka Wojtowicz-Vosloo, Ava DuVernay, Eliz..."
100,Director : Female,W,"7622, 7623, 7650, 7667, 7673, 7681","Lilly Wachowski, Lana Wachowski, Olivia Wilde,..."
101,Director : Female,X,,
102,Director : Female,Y,7680,Naoko Yamada
